<a href="https://colab.research.google.com/github/purnimapatel/Comparing-of-Deep_learning-Neural-Network/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
#import the libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Dense, Dropout


In [3]:
#opening the zip-file
from zipfile import ZipFile
file_name="UCI HAR Dataset.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("DONE")

DONE


In [4]:
#those are separate normalised input features for the neural network
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]
# Output classes to learn how to classify
LABELS = [
    "WALKING",
    "WALKING_UPSTAIRS",
    "WALKING_DOWNSTAIRS",
    "SITTING",
    "STANDING",
    "LAYING"
]

In [5]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load_signals
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/UCI HAR Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        #/content/UCI HAR Dataset/train/Inertial Signals
        #/content/UCI HAR Dataset/train/Inertial Signals/body_acc_x_train.txt
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [6]:
def load_y(subset):
    filename = f'/content/UCI HAR Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

In [8]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, Y_train, Y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    Y_train, Y_test = load_y('train'), load_y('test')

    return X_train, Y_train, X_test,  Y_test

In [12]:
# Loading the train and test data
X_train, Y_train, X_test,  Y_test = load_data()

In [13]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(7352, 128, 9)
(7352, 6)
(2947, 128, 9)
(2947, 6)


In [14]:
from sklearn.base import BaseEstimator, TransformerMixin
class scaling_tseries_data(BaseEstimator, TransformerMixin):
    from sklearn.preprocessing import StandardScaler
    def __init__(self):
        self.scale = None

    def transform(self, X):
        temp_X1 = X.reshape((X.shape[0] * X.shape[1], X.shape[2]))
        temp_X1 = self.scale.transform(temp_X1)
        return temp_X1.reshape(X.shape)

    def fit(self, X):
        # remove overlaping
        remove = int(X.shape[1] / 2)
        temp_X = X[:, -remove:, :]
        # flatten data
        temp_X = temp_X.reshape((temp_X.shape[0] * temp_X.shape[1], temp_X.shape[2]))
        scale = StandardScaler()
        scale.fit(temp_X)
        self.scale = scale
        return self

In [15]:
Scale = scaling_tseries_data()
Scale.fit(X_train)
X_train_sc = Scale.transform(X_train)
X_test_sc = Scale.transform(X_test)

In [17]:
print('Scaled_X_train',X_train_sc.shape)
print('Scaled_X_test',X_test_sc.shape)

Scaled_X_train (7352, 128, 9)
Scaled_X_test (2947, 128, 9)


**Base_Model of CNN**

In [18]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu',kernel_initializer='he_uniform',input_shape=(128,9)))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu',kernel_initializer='he_uniform'))
model.add(Dropout(0.6))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 126, 32)           896       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 124, 32)           3104      
_________________________________________________________________
dropout (Dropout)            (None, 124, 32)           0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 62, 32)            0         
_________________________________________________________________
flatten (Flatten)            (None, 1984)              0         
_________________________________________________________________
dense (Dense)                (None, 50)                99250     
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 3

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
model.fit(X_train_sc,Y_train, epochs=30, batch_size=16,validation_data=(X_test_sc, Y_test), verbose=1)

Epoch 1/30
460/460 [==============================] - 4s 9ms/step - loss: 0.5256 - accuracy: 0.8203 - val_loss: 0.3470 - val_accuracy: 0.9019
Epoch 2/30
460/460 [==============================] - 4s 8ms/step - loss: 0.1372 - accuracy: 0.9426 - val_loss: 0.3372 - val_accuracy: 0.8911
Epoch 3/30
460/460 [==============================] - 4s 9ms/step - loss: 0.1119 - accuracy: 0.9523 - val_loss: 0.2824 - val_accuracy: 0.8979
Epoch 4/30
460/460 [==============================] - 4s 9ms/step - loss: 0.1030 - accuracy: 0.9539 - val_loss: 0.2734 - val_accuracy: 0.9046
Epoch 5/30
460/460 [==============================] - 4s 8ms/step - loss: 0.0967 - accuracy: 0.9606 - val_loss: 0.2534 - val_accuracy: 0.9087
Epoch 6/30
460/460 [==============================] - 4s 8ms/step - loss: 0.0875 - accuracy: 0.9604 - val_loss: 0.2662 - val_accuracy: 0.9111
Epoch 7/30
460/460 [==============================] - 4s 8ms/step - loss: 0.0917 - accuracy: 0.9614 - val_loss: 0.2618 - val_accuracy: 0.9152
Epoch 

In [22]:
Y_pred = model.predict(X_test_sc)

In [23]:
pred = np.argmax(Y_pred,axis = 1) 
Y_actual = np.argmax(Y_test,axis = 1)

In [25]:
confusion_matrix(Y_actual, pred)

array([[489,   1,   6,   0,   0,   0],
       [  7, 444,  20,   0,   0,   0],
       [  4,   1, 415,   0,   0,   0],
       [  3,  10,   0, 407,  69,   2],
       [  5,   4,   0,  99, 424,   0],
       [  0,  18,   0,   0,   0, 519]])

In [26]:
print(classification_report(Y_actual, pred))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97       496
           1       0.93      0.94      0.94       471
           2       0.94      0.99      0.96       420
           3       0.80      0.83      0.82       491
           4       0.86      0.80      0.83       532
           5       1.00      0.97      0.98       537

    accuracy                           0.92      2947
   macro avg       0.92      0.92      0.92      2947
weighted avg       0.92      0.92      0.92      2947



**Regularization and Normalization**

In [27]:
from keras.regularizers import l2,l1
import keras
from keras.layers import BatchNormalization

In [28]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu',kernel_initializer='he_uniform',
                 kernel_regularizer=l2(0.1),input_shape=(128,9)))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu',kernel_regularizer=l2(0.06),kernel_initializer='he_uniform'))
model.add(Dropout(0.65))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 126, 32)           896       
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 124, 16)           1552      
_________________________________________________________________
dropout_1 (Dropout)          (None, 124, 16)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 62, 16)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 992)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                31776     
_________________________________________________________________
dense_3 (Dense)              (None, 6)                

In [30]:
import math
adam = keras.optimizers.Adam(lr=0.001)
rmsprop = keras.optimizers.RMSprop(lr=0.001)
def step_decay(epoch):
    return float(0.001 * math.pow(0.6, math.floor((1+epoch)/10)))
from keras.callbacks import LearningRateScheduler
lrate = LearningRateScheduler(step_decay)
callbacks_list = [lrate]

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [31]:
model.fit(X_train_sc,Y_train, epochs=30, batch_size=16,validation_data=(X_test_sc, Y_test), verbose=1)

Epoch 1/30
460/460 [==============================] - 3s 7ms/step - loss: 4.2121 - accuracy: 0.7399 - val_loss: 1.5692 - val_accuracy: 0.8293
Epoch 2/30
460/460 [==============================] - 3s 6ms/step - loss: 0.7153 - accuracy: 0.9165 - val_loss: 0.6376 - val_accuracy: 0.8860
Epoch 3/30
460/460 [==============================] - 3s 6ms/step - loss: 0.3843 - accuracy: 0.9234 - val_loss: 0.5546 - val_accuracy: 0.8558
Epoch 4/30
460/460 [==============================] - 3s 6ms/step - loss: 0.2986 - accuracy: 0.9321 - val_loss: 0.5078 - val_accuracy: 0.8843
Epoch 5/30
460/460 [==============================] - 3s 6ms/step - loss: 0.2771 - accuracy: 0.9340 - val_loss: 0.4809 - val_accuracy: 0.8741
Epoch 6/30
460/460 [==============================] - 3s 6ms/step - loss: 0.2594 - accuracy: 0.9359 - val_loss: 0.4550 - val_accuracy: 0.8785
Epoch 7/30
460/460 [==============================] - 3s 6ms/step - loss: 0.2501 - accuracy: 0.9380 - val_loss: 0.4540 - val_accuracy: 0.8823
Epoch 

In [32]:
Y_pred = model.predict(X_test_sc)

In [33]:
pred = np.argmax(Y_pred,axis = 1) 
Y_actual = np.argmax(Y_test,axis = 1)

In [34]:
confusion_matrix(Y_actual, pred)

array([[466,   8,  22,   0,   0,   0],
       [ 37, 404,  30,   0,   0,   0],
       [ 19,   2, 399,   0,   0,   0],
       [  1,   9,   0, 380, 101,   0],
       [  0,   0,   0,  57, 475,   0],
       [  0,  27,   0,   5,   0, 505]])

In [35]:
print(classification_report(Y_actual, pred))

              precision    recall  f1-score   support

           0       0.89      0.94      0.91       496
           1       0.90      0.86      0.88       471
           2       0.88      0.95      0.92       420
           3       0.86      0.77      0.81       491
           4       0.82      0.89      0.86       532
           5       1.00      0.94      0.97       537

    accuracy                           0.89      2947
   macro avg       0.89      0.89      0.89      2947
weighted avg       0.89      0.89      0.89      2947

